In [1]:
import os
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document


In [30]:
import dotenv
dotenv.load_dotenv()

base_path = os.getenv("BASE_PATH", ".")
folder_path = os.path.join(base_path,"html_files")

for idx, file_name in enumerate(os.listdir(folder_path)):
    if file_name.endswith(".html"):
        file_path = os.path.join(folder_path, file_name)
        loader = UnstructuredHTMLLoader(file_path)
        documents = loader.load()
        if len(file_name.split('_')) != 6:
            print(file_name,)
        _,_, bc, bt, st, _ = file_name.split('_')
        
        
        for doc in documents:
            remove_first_line = '\n'.join(doc.page_content.split('\n')[1:])
            final_content = f"과목명: {bc}\n"
            final_content += f"주제: {bt}\n"
            final_content += f"소주제: {st}\n\n"
            final_content += remove_first_line
            doc.page_content = final_content
            doc.metadata['file_name'] = file_name
            doc.metadata['chapter'] = bc
            doc.metadata['big_title'] = bt
            doc.metadata['small_title'] = st
        
        if idx == 0:
            all_documents = documents
        else:
            all_documents.extend(documents)


In [32]:

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)
vectorstore = Chroma(
    collection_name="html_docs",
    embedding_function=embedding,
    persist_directory="chroma_db",
)



In [33]:
for i in range(0, len(all_documents), 100):
    print(f"Adding documents {i} to {i+100} to vectorstore")
    batch = all_documents[i:i+100]
    vectorstore.add_documents(batch)

Adding documents 0 to 100 to vectorstore
Adding documents 100 to 200 to vectorstore
Adding documents 200 to 300 to vectorstore
Adding documents 300 to 400 to vectorstore
Adding documents 400 to 500 to vectorstore
Adding documents 500 to 600 to vectorstore
Adding documents 600 to 700 to vectorstore
Adding documents 700 to 800 to vectorstore
Adding documents 800 to 900 to vectorstore
Adding documents 900 to 1000 to vectorstore
Adding documents 1000 to 1100 to vectorstore
Adding documents 1100 to 1200 to vectorstore
Adding documents 1200 to 1300 to vectorstore
Adding documents 1300 to 1400 to vectorstore
Adding documents 1400 to 1500 to vectorstore
Adding documents 1500 to 1600 to vectorstore


In [45]:
query = ["심부전으로 입원한 환자에게 처음 시행해야 할 처치는?",
"수술 후 발생 가능한 DVT 예방 방법은?",
"분만 중 자궁수축 억제제 사용 기준은?",
"조기진통 시 자궁 억제약 처방 기준?",
"열성 경련 1차 대응은?",
"뇌전증 환자의 약 조절 기준은?",
"전자간증 기준 혈압 수치는?",
"심부전으로 입원한 환자의 초기 평가 항목은?",
"고관절 골절 수술 전 체크리스트는?",
"당뇨병성 케톤산증 치료 단계별 접근 방법은?",
"소아 고열 진단 순서는?"]

retriever = vectorstore.as_retriever(
    search_kwargs={"k":5})
print('|query|검색된 문서들|')
print('|---|---|')

all_result = {}
for q in query:
    print(f"|{q}", end='|')
    results = retriever.invoke(q)
    for doc in results:
        print(f"{' '.join(doc.metadata['chapter'].split('.')[1:]).strip()}_{doc.metadata['big_title']}_{doc.metadata['small_title']}", end='<br>')
    print("|")
    all_result[q] = results

|query|검색된 문서들|
|---|---|
|심부전으로 입원한 환자에게 처음 시행해야 할 처치는?|응급의학_2. 외상 환자의 응급처치_1. 응급처치<br>외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperative VTE prophylaxis)<br>부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)<br>외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인지 및 기본 소생술(BLS)<br>외과_5. 수술 전 처치_7. 수술 전 계통별 평가 5- 내분비계 평가<br>|
|수술 후 발생 가능한 DVT 예방 방법은?|순환기_19. 말초혈관질환_7. 심부정맥혈전증 (DVT)<br>외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperative VTE prophylaxis)<br>순환기_19. 말초혈관질환_4. 급성 동맥 폐색<br>외과_5. 수술 전 처치_3. 수술 전 계통별 평가 1- 심혈관계 평가<br>외과_5. 수술 전 처치_8. 수술 전 계통별 평가 6- 혈액학적 평가<br>|
|분만 중 자궁수축 억제제 사용 기준은?|순환기_21. 실신_3. 기립성 저혈압<br>산과_23. 임신 중 고혈압 질환_6. 전자간증 치료<br>순환기_17. 고혈압_3. 고혈압 치료<br>외과_3. 수혈_4. 동결침전제제 (Cryoprecipitate) <br>신장_12. 그외 질환들_1. 부종<br>|
|조기진통 시 자궁 억제약 처방 기준?|부인과_16. 자궁 체부 양성 질환_2. 자궁샘근육증<br>산과_5. 분만 관리 1- 정상 분만_2. 정상 분만의 요소 2- Power - 자궁의 수축력 요소 (분만진통)<br>산과_23. 임신 중 고혈압 질환_2. 임신 중 사용하는 항고혈압 약제<br>산과_21. 조기 산후 출혈_2. 자궁이완증(자궁근육무력증, uterine atony)<br>산과_14. 조산_3. 조기산통 (Preterm Labor, PTL)<br>|
|열성 경련 1차 

In [70]:
def invoke_result(query):
    results = retriever.invoke(query)
    print("query:", query)
    for doc in results:
        print(doc.metadata['file_name'][13:-5].strip())
    print('\n')
invoke_result("심부전")
invoke_result("심부전으로 입원한 환자에게 처음 시행해야 할 처치는?")
invoke_result("심부전으로 입원한 환자의 초기 평가 항목은?")
invoke_result("심부전 환자의 치료는?")

query: 심부전
순환기_11. 심부전_4. 우심부전_79
순환기_11. 심부전_3. 급성 심부전_80
소아과 각론_21. 심혈관 질환_13. 심부전_781
순환기_11. 심부전_1. 심부전 개요_81
순환기_11. 심부전_2. 만성 심부전_82


query: 심부전으로 입원한 환자에게 처음 시행해야 할 처치는?
응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperative VTE prophylaxis)_480
부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)_1424
외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인지 및 기본 소생술(BLS)_1430
외과_5. 수술 전 처치_7. 수술 전 계통별 평가 5- 내분비계 평가_475


query: 심부전으로 입원한 환자의 초기 평가 항목은?
순환기_11. 심부전_1. 심부전 개요_81
응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
외과_10. 외상_14. 전문 외상 처치술 (ATLS)_1472
외과_5. 수술 전 처치_2. 수술 전 위험도 평가_473
순환기_11. 심부전_4. 우심부전_79


query: 심부전 환자의 치료는?
순환기_11. 심부전_4. 우심부전_79
순환기_11. 심부전_3. 급성 심부전_80
순환기_11. 심부전_2. 만성 심부전_82
이비인후과_2. 비과_3. 비부동염_1211
정신과_2. 조현병 스펙트럼 및 기타 정신병적 장애_1. 조현병_878


